### Problem Statement
Create Team wise Task Pendency report, share the summary in automailers and update the same on googlesheet through API

### Data Preprocessing - 
1.Read all files  
2.Clean up data to match column names as data types  
3.Replace null values with 0 or 'NA' depending on the datatype  

In [1]:
import pandas as pd
Forward_3PL=pd.read_excel(r'Report_1.xlsx')
MarketPlace=pd.read_excel(r'Report_2.xlsx')
Transport_ML=pd.read_excel(r'Report_3.xlsx')
Forward_ML=pd.read_excel(r'Report_4.xlsx')
Non_returnable_Tasks=pd.read_excel(r'Report_5.xlsx')
Pickup_3PL=pd.read_excel(r'Report_6.xlsx')
Pickup_ML=pd.read_excel(r'Report_7.xlsx')
Master_3PL=pd.read_excel(r'Report_8.xlsx')
Master_ML=pd.read_excel(r'Report_9.xlsx')
Task_Others=pd.read_excel(r'Report_10.xlsx')
taskdetails=pd.read_csv(r'Report_11.gz')
Warehouse=pd.read_excel(r'Report-12.xlsx')


# In[2]:

Pickup_3PL.rename(columns = {'Category':'Category Level 1','Order Number':'Order_ID','Created Date':'Task Created Date','Return ID':'Entity_Id'}, inplace = True) 
Pickup_ML.rename(columns = {'Category':'Category Level 1','Order Number':'Order_ID','Created Date':'Task Created Date','Return ID':'Entity_Id'}, inplace = True) 
Transport_ML.rename(columns = {'Order ID':'Order_ID','Packet Id':'Entity_Id'}, inplace = True) 
Forward_ML.rename(columns = {'Order ID':'Order_ID','Packet Id':'Entity_Id'}, inplace = True) 
Forward_3PL.rename(columns = {'Order ID':'Order_ID','Packet Id':'Entity_Id'}, inplace = True) 
Task_Others.rename(columns = {'Created Date':'Task Created Date','Task Id':'Task ID'}, inplace = True) 
Master_3PL.rename(columns = {'Created Date':'Task Created Date','ReleaseId':'Entity_Id','Task Id':'Task ID'}, inplace = True) 
Master_ML.rename(columns = {'Created Date':'Task Created Date','ReleaseId':'Entity_Id','Task Id':'Task ID'}, inplace = True) 
Non_returnable_Tasks.rename(columns = {'Created Date':'Task Created Date','Task Id':'Task ID'}, inplace = True) 
MarketPlace.rename(columns = {'Date Created':'Task Created Date','Order ID':'Entity_Id'}, inplace = True) 
Warehouse.rename(columns = {'Created Date':'Task Created Date','Order ID':'Order_ID','ReleaseId':'Entity_Id','Task Category 3':'Category Level 3','Category':'Category Level 1','Sub Category':'Category Level 2'}, inplace = True) 


# In[3]:

ar=['MarketPlace','ML','WH and Logistics','RETURN','Pick-Up','Dispute','3PL']


# In[4]:

temp=[Task_Others[~((Task_Others['Category Level 3'].isin(['Not Packed','Not Packed_WH']))& (Task_Others['Task Status'].isin(['Information Provided','Open','Re-opened'])))],
      Non_returnable_Tasks]
Non_Others=pd.concat(temp)


MarketPlace.loc[~MarketPlace['Category Level 3'].isnull(),'Order_ID']=MarketPlace['Entity_Id']
MarketPlace.loc[~MarketPlace['Category Level 3'].isnull(),'SH']='MarketPlace_Not_Packed'
Warehouse.loc[~Warehouse['Category Level 3'].isnull(),'SH']='Warehouse'
Warehouse.loc[~Warehouse['Category Level 3'].isnull(),'Tracking Number']='Not Applicable'
MarketPlace.loc[~MarketPlace['Category Level 3'].isnull(),'Tracking Number']='Not Applicable'
Warehouse['Order_ID']=Warehouse['Order_ID'].astype(str)

Warehouse['Entity_Id']=Warehouse['Entity_Id'].astype(str)

col=['SH','Category Level 1','Category Level 2','Category Level 3','Order_ID','Task ID'
         ,'Task Status','Tenant ID','Tracking Number','Entity_Id','Task Created Date']

Pre_packed=pd.concat([MarketPlace[MarketPlace['Category Level 3'].isin(['Not Packed','Not Packed_WH'])][col],Warehouse[Warehouse['Category Level 3'].isin(['Not Packed','Not Packed_WH'])][col]])
Pre_packed['Order_ID']=Pre_packed['Order_ID'].astype(str)

Non_Others.loc[~(Non_Others['Category Level 2'].isnull()),'Entity_Id'] = 0
Non_Others.loc[~(Non_Others['Category Level 2'].isnull()),'SH'] = 'Others'

Non_Others.loc[~(Non_Others['Category Level 2'].isnull()),'Entity_Id'] = Non_Others['ReleaseId']
Non_Others.loc[(Non_Others['Category Level 2'].isin(['Non-returnable products'])),'SH'] = 'Non-Returnable'

Non_Others.loc[(Non_Others['Category Level 2'].isin(['WIP/Packed more than 24 hours'])),'Category Level 3'] = 'WIP/Packed more than 24 hours'

Transport_ML.loc[~(Transport_ML['Category Level 3'].isnull()),'Category Level 1'] = 'ML'
Transport_ML.loc[(Transport_ML['Category Level 2'].isin(['WIP/Packed more than 24 hours'])),'Category Level 1'] = 'WH and Logistics'

Transport_ML.loc[(Transport_ML['Category Level 3'].isin(['WIP/Packed more than 72 hours'])),'Category Level 1'] = 'MarketPlace'
Transport_ML.loc[~(Transport_ML['Category Level 2'].isnull()),'SH'] = 'ML_Transport'
Transport_ML.loc[(Transport_ML['Category Level 2'].isin(['WIP/Packed more than 24 hours'])),'Category Level 3'] = 'WIP/Packed more than 24 hours'


# In[6]:


Forward_ML.loc[~(Forward_ML['Category Level 3'].isnull()),'Category Level 1'] = 'ML'
Forward_ML.loc[(Forward_ML['Category Level 2'].isin(['Reinvestigation Request'])),'Category Level 1'] = 'Dispute'
Forward_ML.loc[~(Forward_ML['Category Level 3'].isnull()),'SH'] = 'ML_Forward'


# In[8]:


Pickup_3PL.loc[(Pickup_3PL['Category Level 3'].isin(['Delayed more than 3 Business days'])),'Category Level 2'] = 'Delayed'
Pickup_3PL.loc[(Pickup_3PL['Category Level 3'].isin(['Pick Up Approvals'])),'Category Level 2'] = 'Approvals'
Pickup_3PL.loc[(Pickup_3PL['Category Level 3'].isin(['Disputing pick up attempt'])),'Category Level 2'] = 'Disputed pick up'
Pickup_3PL.loc[(Pickup_3PL['Category Level 3'].isin(['Picked up not delivered to WH'])),'Category Level 2'] = 'Picked Up Refund Pending'
Pickup_3PL.loc[(Pickup_3PL['Category Level 3'].isin(['Exchange Approvals'])),'Category Level 2'] = 'Approvals'
Pickup_3PL.loc[~(Pickup_3PL['Category Level 3'].isnull()),'Category Level 1'] = 'Pick-Up'
Pickup_3PL.loc[~(Pickup_3PL['Category Level 3'].isnull()),'SH'] = 'Reverse_3PL'


# In[9]:


Pickup_ML.loc[(Pickup_ML['Category Level 3'].isin(['Delayed more than 3 Business days'])),'Category Level 2'] = 'Delayed'
Pickup_ML.loc[(Pickup_ML['Category Level 3'].isin(['Pick Up Approvals','Exchange Approvals'])),'Category Level 2'] = 'Approvals'
Pickup_ML.loc[(Pickup_ML['Category Level 3'].isin(['Disputing pick up attempt'])),'Category Level 2'] = 'Disputed pick up'
Pickup_ML.loc[(Pickup_ML['Category Level 3'].isin(['Picked up not delivered to WH'])),'Category Level 2'] = 'Picked Up Refund Pending'
Pickup_ML.loc[~(Pickup_ML['Category Level 3'].isnull()),'Category Level 1'] = 'Pick-Up'
Pickup_ML.loc[~(Pickup_ML['Category Level 3'].isnull()),'SH'] = 'Reverse_ML'
Forward_3PL.loc[~(Forward_3PL['Category Level 3'].isnull()),'SH'] = '3PL_Forward'

Non_Others.loc[~(Non_Others['Category Level 3'].isnull()),'Order_ID'] = 'Not Applicable'
Non_Others.loc[~(Non_Others['Category Level 3'].isnull()),'Tracking Number'] = 'Not Applicable'

Master_3PL.loc[~(Master_3PL['Category Level 3'].isnull()),'Order_ID'] = 'Not Applicable'
Master_3PL.loc[~(Master_3PL['Category Level 3'].isnull()),'Tracking Number'] = 'Not Applicable'
Master_3PL.loc[~(Master_3PL['Category Level 3'].isnull()),'SH'] = 'Others'

Master_ML.loc[~(Master_ML['Category Level 3'].isnull()),'Order_ID'] = 'Not Applicable'
Master_ML.loc[~(Master_ML['Category Level 3'].isnull()),'Tracking Number'] = 'Not Applicable'
Master_ML.loc[~(Master_ML['Category Level 3'].isnull()),'SH'] = 'Others'



C:\Users\300024402\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\300024402\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



### Combine reports and create various subsets of data for teams handling Forward and Reverse categories

In [4]:
# In[10]:


columns=['SH','Category Level 1','Category Level 2','Category Level 3','Order_ID','Task ID'
         ,'Task Status','Tenant ID','Tracking Number','Entity_Id','Task Created Date']
Transport_ML_1=Transport_ML[columns]
Forward_ML_1=Forward_ML[columns]
Forward_3PL_1=Forward_3PL[columns]
Pickup_3PL_1=Pickup_3PL[columns]
Pickup_ML_1=Pickup_ML[columns]
# Task_Others_1=Task_Others[columns]
Master_3PL_1=Master_3PL[columns]
Master_ML_1=Master_ML[columns]
Non_Others_1=Non_Others[columns]

# Change datatype
import numpy as np
Transport_ML_1['Entity_Id']=Transport_ML_1['Entity_Id'].fillna(0).astype(np.int64)
Forward_ML_1['Entity_Id']=Forward_ML_1['Entity_Id'].fillna(0).astype(np.int64)
Forward_3PL_1['Entity_Id']=Forward_3PL_1['Entity_Id'].fillna(0).astype(np.int64)
Pickup_3PL_1['Entity_Id']=Pickup_3PL_1['Entity_Id'].fillna(0).astype(np.int64)
Pickup_ML_1['Entity_Id']=Pickup_ML_1['Entity_Id'].fillna(0).astype(np.int64)
# Task_Others_1['Entity_Id']=Task_Others_1['Entity_Id'].fillna(0).astype(np.int64)
Master_3PL_1['Entity_Id']=Master_3PL_1['Entity_Id'].fillna(0).astype(np.int64)
Master_ML_1['Entity_Id']=Master_ML_1['Entity_Id'].fillna(0).astype(np.int64)
# Task_Others['Style Id']=Task_Others['Style Id'].fillna(0).astype(np.int64)
Non_Others_1['Entity_Id']=Non_Others_1['Entity_Id'].fillna(0).astype(np.int64)


# Concatenate dataframes
frames_r=[Pickup_3PL_1,Pickup_ML_1]
result_r=pd.concat(frames_r)
frames_f=[Transport_ML_1,Forward_ML_1,Forward_3PL_1]
result_f=pd.concat(frames_f)
frames=[result_f,result_r]
result_1=pd.concat(frames)


# Remove tasks in from DF A if it is in DF B
task_check=[]
task_check=result_1['Task ID']
task_check_1=[]
task_check_1=Non_Others_1['Task ID']
Non_Others_1=Non_Others_1[~(Non_Others_1['Task ID'].isin(task_check))]
Non_Others_1.drop_duplicates()
Master_ML_1=Master_ML_1[~(Master_ML_1['Task ID'].isin(task_check))]
Master_3PL_1=Master_3PL_1[~(Master_3PL_1['Task ID'].isin(task_check))]
rame=[result_1,Master_ML_1,Master_3PL_1,Non_Others_1, Pre_packed]
arr2=[Master_ML_1,Master_3PL_1,Non_Others_1, Pre_packed]
master_concat=pd.concat(arr2)
result_v1=pd.concat(rame)



result_v1['Entity_Id']=result_v1['Entity_Id'].astype(str)
result_v1["concat"]=result_v1["Category Level 1"].map(str)+'-'+result_v1["Category Level 2"].map(str)+'-'+result_v1["Category Level 3"].map(str)

#Filter only Comp A tasks
result=result_v1.loc[((result_v1['Tenant ID'])==1234)]


#Remove IST from created date and convert to string
from datetime import datetime
result["Task Created Date"] = [sub.replace('IST ', '') for sub in result["Task Created Date"]]

#Calculate ageing of open tasks
t=result["Task Created Date"].tolist()
i=0
length=len(result["Task Created Date"])
ageing_in_hours=[]
c=datetime.now()
while i < length: 
    j=(c-datetime.strptime(t[i],"%a %b %d %H:%M:%S %Y") ).total_seconds()/(86400) 
    ageing_in_hours.append(round(j,2))
    i+=1
result["Ageing_in_days"]=ageing_in_hours


# Create ageing busckets basis ageing calculated in previous step
result.loc[(result['Ageing_in_days']>10),'Ageing_bucket'] = 'Days10+'
result.loc[(result['Ageing_in_days']<=10),'Ageing_bucket'] = '9 to 10'
result.loc[(result['Ageing_in_days']<=9),'Ageing_bucket'] = '8 to 9'
result.loc[(result['Ageing_in_days']<=8),'Ageing_bucket'] = '7 to 8'
result.loc[(result['Ageing_in_days']<=7),'Ageing_bucket'] = '6 to 7'
result.loc[(result['Ageing_in_days']<=6),'Ageing_bucket'] = '5 to 6'
result.loc[(result['Ageing_in_days']<=5),'Ageing_bucket'] = '4 to 5'
result.loc[(result['Ageing_in_days']<=4),'Ageing_bucket'] = '3 to 4'
result.loc[(result['Ageing_in_days']<=3),'Ageing_bucket'] = '2 to 3'
result.loc[(result['Ageing_in_days']<=2),'Ageing_bucket'] = '1 to 2'
result.loc[(result['Ageing_in_days']<=1),'Ageing_bucket'] = '0 to 1'


result.loc[(result['Ageing_in_days']>10),'Ageing_bucket_EORS'] = 'Days10+'
result.loc[(result['Ageing_in_days']<=10),'Ageing_bucket_EORS'] = '8 to 10'
result.loc[(result['Ageing_in_days']<=7),'Ageing_bucket_EORS'] = '4 to 7'
result.loc[(result['Ageing_in_days']<=3),'Ageing_bucket_EORS'] = '0 to 3'


# In[22]:


# Subset data basis the given condition 
ar_r3=['Defective post usage','PPS-Refund-IMPS','PPS-Refund-NEFT','NEFT-Transfer-NEFT transfer','Self-Shipped No Update from WH','Missing Item in Return Packet','Rejected Reshipment Pending','QA failed by DC/WH','No update from WH','Defective product received','Wrong product received','Price tags missing','Used product received','MRP mismatch','Article ID mismatch']
ar_f3=['Re- Investigate POD','Disputing signature','Full Item missing','Partial Item Missing']
ar_f1=['3PL','ML','MarketPlace','WH and Logistics']
ar_r1=['Pick-Up','ROH']


# In[23]:


#Packet_ids from seperate reports
Fwd_entity1 = (result_f[((result_f['Category Level 1'].isin(ar_f1))|(result_f['Category Level 3'].isin(ar_f3)))&(~result_f['Entity_Id'].isnull())&(~(result_f['Entity_Id']==0))]['Entity_Id']).unique().tolist()


#Release_ids from Master reports for fwd
Fwd_entity2 = (master_concat[((master_concat['Category Level 1'].isin(ar_f1))|(master_concat['Category Level 3'].isin(ar_f3)))&(~master_concat['Entity_Id'].isnull())&(~master_concat['Category Level 3'].isin(['Not Packed', 'Not Packed_WH']))&(~(master_concat['Entity_Id']==0))]['Entity_Id']).unique().tolist()

Fwd_entity3 = (master_concat[(master_concat['Category Level 3'].isin(['Not Packed', 'Not Packed_WH']))&(~master_concat['Entity_Id'].isnull())&(~(master_concat['Entity_Id']==0))]['Entity_Id']).unique().tolist()

#Return_ids from seperate reports
Rev_entity1 = (result_r[(result_r['Category Level 1'].isin(ar_r1) | result_r['Category Level 3'].isin(ar_r3))&(~result_r['Entity_Id'].isnull())&(~(result_r['Entity_Id']==0))]['Entity_Id']).unique().tolist()


#Release_ids from Master reports for reverse
Rev_entity2 =(master_concat[(master_concat['Category Level 1'].isin(ar_r1) | master_concat['Category Level 3'].isin(ar_r3))&(~master_concat['Entity_Id'].isnull())&(~(master_concat['Entity_Id']==0))]['Entity_Id']).unique().tolist()


C:\Users\300024402\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\300024402\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\300024402\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [11]:
# In[29]:
taskdetails['inapp_interactions']=taskdetails['inapp_interactions'].fillna(0).astype(np.int64)
taskdetails['Intercation_count_call']=taskdetails['Intercation_count_call'].fillna(0).astype(np.int64)
taskdetails['Total_interactions']=taskdetails['inapp_interactions']+taskdetails['Intercation_count_call']


taskdetails.loc[(taskdetails['Total_interactions']>3),'Interaction_Bucket'] = '3+'
taskdetails.loc[(taskdetails['Total_interactions']==3),'Interaction_Bucket'] = '3'
taskdetails.loc[(taskdetails['Total_interactions']==2),'Interaction_Bucket'] = '2'
taskdetails.loc[(taskdetails['Total_interactions']<=1),'Interaction_Bucket'] = '0 to 1'

### Fetch futher details from database based on issue ids extracted in earlier step

In [12]:
import pymysql.cursors

order_release_id="""
select
zipcode,
ol.created_on,
packet_id_fk as Entity_Id,
packet_id_fk as packet_return_id,
ol.courier_code,
ol.status_code,
pk.final_amount,
ol.payment_method as payment_mode,
source_wh_id
from order_line ol 
left join order_release_VIEW ord
on ol.store_order_id=ord.store_order_id
left join (select packet_id_fk as packet,
sum(final_amount) as final_amount
from order_line
where packet_id_fk in ('""" +  "','".join(map(str, Fwd_entity1)) + """')
group by 1)pk
on ol.packet_id_fk=pk.packet
where ol.packet_id_fk in ('""" +  "','".join(map(str, Fwd_entity1)) + """') 
group by 1,2,3,4,5
"""

connection = pymysql.connect(host='host',
                            user='user',
                            password='pswrd',                             
                            db='db',
                            charset='some charset',
                            cursorclass=pymysql.cursors.DictCursor)
order_release_id_df1=pd.read_sql(order_release_id,connection)

In [16]:

import pymysql.cursors

order_release_id="""
select
zipcode,
ol.created_on,
order_release_id_fk as Entity_Id,
packet_id_fk as packet_return_id,
ol.courier_code,
ol.status_code,
pk.final_amount,
ol.payment_method as payment_mode,
source_wh_id
from order_line ol 
left join order_release_VIEW ord
on ol.store_order_id=ord.store_order_id
left join (select packet_id_fk as packet,
sum(final_amount) as final_amount
from order_line
where order_release_id_fk in ('""" +  "','".join(map(str, Fwd_entity2)) + """')
group by 1)pk
on ol.packet_id_fk=pk.packet
where ol.order_release_id_fk in ('""" +  "','".join(map(str, Fwd_entity2)) + """') 
group by 1,2,3,4,5
"""

connection = pymysql.connect(host='host',
                            user='user',
                            password='pswrd',                             
                            db='db',
                            charset='some charset',
                            cursorclass=pymysql.cursors.DictCursor)
order_release_id_df2=pd.read_sql(order_release_id,connection)

In [17]:
order_release_id="""
select
zipcode,
ol.created_on,
ol.store_order_id as Entity_Id,
packet_id_fk as packet_return_id,
ol.courier_code,
ol.status_code,
pk.final_amount,
ol.payment_method as payment_mode,
source_wh_id
from order_line ol 
left join order_release_VIEW ord
on ol.store_order_id=ord.store_order_id
left join (select order_release_id_fk as order_release,
sum(final_amount) as final_amount
from order_line
where store_order_id in ('""" +  "','".join(map(str, Fwd_entity3)) + """')
and status_code in ('WP')
group by 1)pk
on ol.order_release_id_fk=pk.order_release
where ol.store_order_id in ('""" +  "','".join(map(str, Fwd_entity3)) + """') 
and ol.status_code in ('WP')
group by 1,2,3,4,5
"""

connection = pymysql.connect(host='host',
                            user='user',
                            password='pswrd',                             
                            db='db',
                            charset='some charset',
                            cursorclass=pymysql.cursors.DictCursor)
order_release_id_df3=pd.read_sql(order_release_id,connection)

In [18]:
order_release_id="""
select
zipcode,
order_release_id_fk as Entity_Id,
packet_id_fk as packet_return_id,
ol.created_on,
ol.courier_code,
ol.status_code,
pk.final_amount,
ol.payment_method as payment_mode,
source_wh_id
from order_line ol
left join order_release_VIEW ord
on ol.store_order_id=ord.store_order_id
left join (select order_release_id_fk as order_release,
final_amount
from order_line
where order_release_id_fk in ('""" +  "','".join(map(str, Fwd_entity3)) + """')
group by 1)pk
on ol.order_release_id_fk=pk.order_release
where ol.order_release_id_fk in ('""" +  "','".join(map(str, Fwd_entity3)) + """')
group by 1,2,3,4,5
"""

connection = pymysql.connect(host='host',
                            user='user',
                            password='pswrd',                             
                            db='db',
                            charset='some charset',
                           cursorclass=pymysql.cursors.DictCursor)
order_release_id_df4=pd.read_sql(order_release_id,connection)

In [20]:
# Changed on 16th Jul

pushed_not_packed="""
SELECT cor.portal_order_release_id AS Entity_Id,
cor.pushed_to_pick_time AS ptp_time
FROM   capture_order_release cor
WHERE  cor.order_release_status NOT IN ('CANCELLED')
AND  cor.portal_order_release_id IN ('""" +  "','".join(map(str, Fwd_entity3)) + """')
"""


connection_3=pymysql.connect(host='host',
                            user='user',
                            password='pswrd',                             
                            db='db',
                            charset='some charset',
                           cursorclass=pymysql.cursors.DictCursor)
pushed_not_packed_df=pd.read_sql(pushed_not_packed,connection_3)

In [21]:
# In[34]:


import pymysql.cursors

returns_data="""

select 
r.zipcode,
rl.created_on,
r.id as Entity_Id, 
return_id as packet_return_id,
r.courier_code,
r.status_code,
final_amount,
refund_mode as payment_mode,
'NA' as source_wh_id
from return_line rl
left join ( 
select 
id,
zipcode,
courier_code,
status_code,
refund_mode,
refund_amount as final_amount
from 
returns_VIEW
)r
on rl.return_id=r.id
where return_id in ('""" +  "','".join(map(str, Rev_entity1)) + """')
order by rl.created_on desc
"""

connection = pymysql.connect(host='host',
                            user='user',
                            password='pswrd',                             
                            db='db',
                            charset='some charset',
                            cursorclass=pymysql.cursors.DictCursor)
Returns_data_df_1=pd.read_sql(returns_data,connection)


In [22]:
# In[35]:


import pymysql.cursors

returns_data="""

select 
r.zipcode,
rl.created_on,
order_release_id as Entity_Id, 
return_id as packet_return_id,
r.courier_code,
r.status_code,
final_amount,
refund_mode as payment_mode,
'NA' as source_wh_id
from return_line rl
left join ( 
select 
id,
zipcode,
courier_code,
status_code,
refund_mode,
refund_amount as final_amount
from 
returns_VIEW
)r
on rl.return_id=r.id
where  order_release_id in ('""" +  "','".join(map(str, Rev_entity2)) + """') and r.status_code not in ('DEC')
order by rl.created_on desc
"""

connection = pymysql.connect(host='host',
                            user='user',
                            password='pswrd',                             
                            db='db',
                            charset='some charset',
                            cursorclass=pymysql.cursors.DictCursor)
Returns_data_df_2=pd.read_sql(returns_data,connection)

In [23]:
# In[36]:

import re 
result["Entity_Id"]=result["Entity_Id"].astype(str)


# In[37]:


Returns_data_df_1.drop_duplicates(inplace = True)
Returns_data_df_2.drop_duplicates(inplace = True)
order_release_id_df1.drop_duplicates(inplace = True)
order_release_id_df2.drop_duplicates(inplace = True)
order_release_id_df3.drop_duplicates(inplace = True)
order_release_id_df4.drop_duplicates(inplace = True)

# In[38]:


Final_v1=()
Returns_data_df_1['Entity_Id']=Returns_data_df_1['Entity_Id'].astype(str)
Returns_data_df_2['Entity_Id']=Returns_data_df_2['Entity_Id'].astype(str)
fr_r=[Returns_data_df_1,Returns_data_df_2]
Final_v1=pd.concat(fr_r)
Final_v1.drop_duplicates(inplace = True)


# In[39]:


order_release_id_df1['Entity_Id']=order_release_id_df1['Entity_Id'].astype(str)
order_release_id_df2['Entity_Id']=order_release_id_df2['Entity_Id'].astype(str)
order_release_id_df3['Entity_Id']=order_release_id_df3['Entity_Id'].astype(str)
order_release_id_df4['Entity_Id']=order_release_id_df4['Entity_Id'].astype(str)
fr_f=[order_release_id_df1,order_release_id_df2, order_release_id_df3,order_release_id_df4]
Final_v2=pd.concat(fr_f)
Final_v2.drop_duplicates(inplace = True)


# In[40]:


fr=[Final_v1,Final_v2]
Final_1=pd.concat(fr)
Final_1["Entity_Id"]=Final_1["Entity_Id"].astype(str)
result['Entity_Id']=result['Entity_Id'].astype(str)
Final_1["final_amount"] = Final_1["final_amount"].fillna(0)


# In[41]:


Final_2 = pd.merge(result,Final_1,  how='left', left_on=['Entity_Id'], right_on = ['Entity_Id'])

C:\Users\300024402\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\300024402\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



### Extract location for all the issues Ids

In [24]:
import os
os.chdir('D:\\Python results')
import pandas as pd
import ddp_new as ddp
import numpy as np
import datetime
import re
QUERY_USER='mail id'
QUERY_OUT_DIR='D:\\Python results'


# In[43]:


Sql="""select pincode,dl.city_group as city
from dim_location dl where cast(pincode as varchar) in('""" +  "','".join(map(str, g)) + """')"""


# In[44]:


QUERY_NAME='location'
print("I am here-1")
Sql = re.sub("%(?![0-9a-fA-F]{2})", "%25", Sql)
Sql = re.sub("\\+", "%2B", Sql)
filename=ddp.start(Sql,QUERY_USER,QUERY_OUT_DIR,QUERY_NAME,0)
print ("I am here -2")

I am here-1
Submitting Query to DDP
Query submission success. QueryId:856985
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Query Status
Query still executing. Sleeping:60 seconds
Checking Quer

In [25]:
location=pd.read_table('D:\\Python resultslocation000.gz',compression='gzip',sep=',',header=None,parse_dates=True,error_bad_lines=False)
location.columns = location.iloc[0]
location = location[1:]

k = Final_2['Tracking Number'].unique().tolist()

In [26]:

order_release_id="""
(select tracking_no as tracking_number,
dc.code as DC,
max(cast(assignment_date as date)) as assignment_date
from
trip_order_assignment toa
left join trip t
on toa.trip_id=t.id  
left join delivery_center dc
on t.delivery_center_id=dc.id
where toa.tracking_no in ('""" +  "','".join(map(str, k)) + """')
group by tracking_no,dc.code)
union
(select ots.tracking_number,
dc.code as DC,
max(cast(ots.last_modified_on as date)) as assignment_date
from
order_to_ship ots
left join delivery_center dc
on ots.delivery_center_id=dc.id
where ots.tracking_number in ('""" +  "','".join(map(str, k)) + """')
group by tracking_number,dc.code)
union
(select rs.tracking_number,
dc.code as DC,
max(cast(rs.last_modified_on as date)) as assignment_date
from
return_shipment rs
left join delivery_center dc
on rs.delivery_center_id = dc.id
where rs.tracking_number in ('""" +  "','".join(map(str, k)) + """')
group by tracking_number,dc.code)
"""
connection = pymysql.connect(host='host',
                            user='user',
                            password='pswrd',                             
                            db='db',
                            charset='some charset',
                            cursorclass=pymysql.cursors.DictCursor)
location_dc=pd.read_sql(order_release_id,connection)

In [27]:
location_dc['row_number']=location_dc.sort_values('assignment_date', ascending = False).groupby('tracking_number').cumcount()

In [28]:
location_dc_1=location_dc[location_dc['row_number']==0]
# print(location_dc.shape)
# print(location_dc_1.shape)
location.rename(columns = {'pincode':'zipcode'}, inplace= True)

In [30]:
location['zipcode']=location['zipcode'].astype(str)
location_dc_1['tracking_number']=location_dc_1['tracking_number'].astype(str)

Final_3=pd.DataFrame()
Final_3 = pd.merge(Final_2,location,  how='left', left_on=['zipcode'], right_on = ['zipcode'])

C:\Users\300024402\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [33]:
pushed_not_packed_df['Entity_Id']=pushed_not_packed_df['Entity_Id'].astype(str)
Final_4=pd.DataFrame()
Final_4 = pd.merge(Final_3,location_dc_1,  how='left', left_on=['Tracking Number'], right_on = ['tracking_number'])

Final_5=pd.DataFrame()
Final_5 = pd.merge(Final_4,taskdetails,  how='left', left_on=['Task ID'], right_on = ['task_id'])
Final_5 = pd.merge(Final_5,pushed_not_packed_df,  how='left', left_on=['Entity_Id'], right_on = ['Entity_Id'])

In [34]:
Final_5.loc[((~Final_5.orderdetails_returnid.isnull())
             &(Final_5.packet_return_id.isnull())
             &(Final_5['SH'].isin(['Reverse_ML, Reverse_3PL'])),
             'packet_return_id' )]=Final_5.orderdetails_returnid

x=Final_5[Final_5.SH.isin(['Reverse_ML', 'Reverse_3PL'])]['packet_return_id'].fillna(0).unique().astype(np.int64).astype(str).tolist()

y=Final_5[Final_5.SH.isin(['ML_Transport', 'ML_Forward','3PL_Forward'])]['packet_return_id'].fillna(0).unique().astype(np.int64).astype(str).tolist()


In [35]:
add_details="""
(select  
source_return_id as packet_return_id,
address_type,
shipment_status,
'' as dc_inscanned_date
from return_shipment rs
where source_return_id in ('""" +  "','".join(map(str, x)) + """'))

union 
(select order_id as packet_return_id, address_type, 'NA' as shipment_status,
max(dc_inscanned_date) as dc_inscanned_date
from order_to_ship ots
left join 
	(
		select source_reference_id , max(received_date) as dc_inscanned_date
		from ml_shipment mls
		where source_reference_id in ('""" +  "','".join(map(str, y)) + """')
        group by 1
	)mls
on ots.order_id=mls.source_reference_id 
where order_id in ('""" +  "','".join(map(str, y)) + """')
group by 1,2,3)


"""


connection = pymysql.connect(host='host',
                            user='user',
                            password='pswrd',                             
                            db='db',
                            charset='charset',
                            cursorclass=pymysql.cursors.DictCursor)
add_details_df=pd.read_sql(add_details,connection)

In [37]:
Final_5.packet_return_id=Final_5.packet_return_id.fillna(0).astype(np.int64).astype(str)

add_details_df.packet_return_id=add_details_df.packet_return_id.fillna(0).astype(np.int64).astype(str)

In [38]:
Final_5=pd.merge(Final_5,add_details_df, how='left', left_on='packet_return_id', right_on='packet_return_id')

In [40]:
Inv_SJIT=[List of IDs]

JIT_FBM=[List of IDs]

MDirect=[List of IDs]

VendorFlex=[List of IDs]

Final_5.loc[((Final_5.source_wh_id.astype(str).isin(Inv_SJIT)),

             'PPMP/Vendor_Flex' )]='Inv_SJIT'

Final_5.loc[((Final_5.source_wh_id.astype(str).isin(JIT_FBM)),

             'PPMP/Vendor_Flex' )]='JIT_FBM'

Final_5.loc[((Final_5.source_wh_id.astype(str).isin(MDirect)),

             'PPMP/Vendor_Flex' )]='MDirect'

Final_5.loc[((Final_5.source_wh_id.astype(str).isin(VendorFlex)),

             'PPMP/Vendor_Flex' )]='VendorFlex'

Final_5.loc[(~(Final_5.dc_inscanned_date.isnull()),

             'Reached_DC' )]='1'

In [41]:
Incident_cat_1=()
Incident_cat_1=Incident_cat[['cat','Issue Type 1','Issue Type 2','Issue Type 3']]

Final_6 = pd.merge(Final_5,Incident_cat_1,how='left', left_on=['concat'], right_on = ['cat'])

# In[55]:


Final_7 =Final_6[~((Final_6['Category Level 1'].isin(['Pick-Up']))&(Final_6['status_code'].isin(['D','C'])))]
Final_7=Final_7.drop_duplicates()


### Expoting the issue ids with teams and their respective categories to Google sheets

In [52]:
SCM_columns=['SH', 'Category Level 1','Category Level 2', 'Category Level 3', 'Task Status','Ageing_bucket','zipcode', 'courier_code',
'status_code','source_wh_id', 'city', 'DC','task_queue','queue_name','escalated_to','escalated_on','Total_interactions', 'Interaction_Bucket','order_created_date','address_type','shipment_status', 'Reached_DC','PPMP/Vendor_Flex','Issue Type 1','Issue Type 2', 'Issue Type 3',
'Reverse Serviceable_flag', 'Forward Serviceability status']

In [54]:
from datetime import date
SCM_Review=Final_8[(~Final_8['SH'].isin(['Others','Non-Returnable']))
                  |(Final_8['Category Level 3'].isin(['Not Packed','Not Packed_WH']))
                  &Final_8['Task Status'].isin(['Re-opened','Open','Information Provided'])]
SCM_Review['created_on']=SCM_Review['created_on'].astype(str)

C:\Users\300024402\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [55]:
SCM_Review_1=SCM_Review.fillna(0).groupby (SCM_columns).agg({'Task ID':pd.Series.nunique}).reset_index()
SCM_Review_1['Pendency_date']=str(date.today())

### Report 1

In [56]:
SAMPLE_SPREADSHEET_ID='1Sheet id'
SAMPLE_RANGE_NAME='Sheet name'
#authorization
gc = py.authorize(service_file='Path//creds.json')

#open the google spreadsheet 'Hard Stop' is the sheet name
sh = gc.open('Sheet Name')

In [57]:
wks_0=sh.worksheet_by_title('Pendency snapshot RD')
wks_0.append_table(SCM_Review_1.fillna(0).values.tolist(),overwrite=False)


### Report 2

In [58]:
SAMPLE_SPREADSHEET_ID='sample spreadsheet'

sh1 = gc.open('Return Pendency AQS')
wks_1=sh1.worksheet_by_title('Task TN')

wks_1.clear()

wks_1.set_dataframe(Final_8[(Final_8['Category Level 3'].isin(['Delayed more than 3 Business days','Disputing pick up attempt']))
       &(Final_8['courier_code']=='ML')
        &(Final_8['SH'].isin(['Reverse_ML']))][['Tracking Number','Task ID']],(1,1), Index=False)


from datetime import datetime
today = datetime.today().strftime('%d-%m-%Y %H:%M:%S')
wks_1.update_value('F1','Last modified on '+today)

### Create Dataframe for daily automailers

In [59]:
Task_category=Final_8.pivot_table(index=['Category Level 3'], values=['Task ID'], columns='Ageing_bucket', aggfunc=pd.Series.nunique).fillna(0).astype(np.int64)
Task_category.columns=Task_category.columns.get_level_values(1)
Task_category=Task_category.rename_axis(['Category Level 3']).reset_index(level=0)
Task_category['Total']=Task_category.sum(numeric_only=True, axis=1)
Task_category.sort_values(by='Total', ascending=False,inplace=True)
Task_category.to_csv('Task_category.csv', index=False)
Task_category=pd.read_csv('Task_category.csv')
Task_category_1=Task_category[~Task_category['Category Level 3'].isin(['Exchange Approvals', "Pick up Approvals'"])][:10]

In [60]:
Incient_category=Final_8.pivot_table(index=['Issue Type 3'], values=['Task ID'], columns='Ageing_bucket', aggfunc=pd.Series.nunique).fillna(0).astype(np.int64)
Incient_category.columns=Incient_category.columns.get_level_values(1)
Incient_category=Incient_category.rename_axis(['Issue Type 3']).reset_index(level=0)
Incient_category['Total']=Incient_category.sum(numeric_only=True, axis=1)
Incient_category.sort_values(by='Total', ascending=False,inplace=True)
Incient_category.to_csv('Incient_category.csv', index=False)
Incient_category=pd.read_csv('Incient_category.csv')
Incient_category_1=Incient_category[~Incient_category['Issue Type 3'].isin(['Exchange Approval', 'Pick up Approval'])][:10]

In [61]:
SH=Final_8.pivot_table(index=['SH'], values=['Task ID'], columns='Ageing_bucket', aggfunc=pd.Series.nunique).fillna(0).astype(np.int64)
SH.columns=SH.columns.get_level_values(1)
SH=SH.rename_axis(['SH']).reset_index(level=0)
SH['Total']=SH.sum(numeric_only=True, axis=1)
SH.to_csv('SH.csv', index=False)
SH=pd.read_csv('SH.csv')
SH.loc['Grand Total']=SH.sum( axis=0)

In [62]:
Task_Count=Final_8[Final_8['Task Status'].isin(['Open','Additional Information Required','Re-opened','Information Provided'])].groupby('Ageing_bucket_EORS').agg({ 'Task ID': pd.Series.nunique}).T.rename_axis(['Ageing']).reset_index()
Task_Count.to_csv('Task_Count.csv', index=False)
Task_Count=pd.read_csv('Task_Count.csv')

Incient_category_EORS=Final_8[Final_8['Task Status'].isin(['Open','Additional Information Required','Re-opened','Information Provided'])].pivot_table(index=['Issue Type 3'], values=['Task ID'], columns='Ageing_bucket_EORS', aggfunc=pd.Series.nunique).fillna(0).astype(np.int64)
Incient_category_EORS.columns=Incient_category_EORS.columns.get_level_values(1)
Incient_category_EORS=Incient_category_EORS.rename_axis(['Issue Type 3']).reset_index(level=0)
Incient_category_EORS['Total']=Incient_category_EORS.sum(numeric_only=True, axis=1)
Incient_category_EORS.sort_values(by='Total', ascending=False,inplace=True)
Incient_category_EORS.to_csv('Incient_category_EORS.csv', index=False)
Incient_category_EORS=pd.read_csv('Incient_category_EORS.csv')
Incient_category_EORS_1=Incient_category_EORS[~Incient_category_EORS['Issue Type 3'].isin(['Exchange Approval', 'Pick up Approval'])][:10]

Task_Count_Interaction=Final_8[Final_8['Task Status'].isin(['Open','Additional Information Required','Re-opened','Information Provided'])].groupby('Interaction_Bucket').agg({'Task ID': pd.Series.nunique}).T.rename_axis(['Interactions']).reset_index()
Task_Count_Interaction.to_csv('Task_Count_Interaction.csv', index=False)
Task_Count_Interaction=pd.read_csv('Task_Count_Interaction.csv')

In [63]:
SH_Open=Final_8[Final_8['Task Status'].isin(['Open'])].pivot_table(index=['SH'], values=['Task ID'], columns='Ageing_bucket', aggfunc=pd.Series.nunique).fillna(0).astype(np.int64)
SH_Open.columns=SH_Open.columns.get_level_values(1)
SH_Open=SH_Open.rename_axis(['SH']).reset_index(level=0)
SH_Open['Total']=SH_Open.sum(numeric_only=True, axis=1)
SH_Open.to_csv('SH_Open.csv', index=False)
SH_Open=pd.read_csv('SH_Open.csv')
SH_Open.loc['Grand Total']=SH_Open.sum( axis=0)

In [64]:
City_split=Final_8.pivot_table(index=['city'], values=['Task ID'], columns='Ageing_bucket', aggfunc=pd.Series.nunique).fillna(0).astype(np.int64)
City_split.columns=City_split.columns.get_level_values(1)
City_split=City_split.rename_axis(['city']).reset_index(level=0)
City_split['Total']=Final_8.groupby("city").agg({ 'Task ID': pd.Series.nunique}).reset_index()['Task ID']
City_split.sort_values(by='Total', ascending=False,inplace=True)
City_split.to_csv('City_split.csv', index=False)
City_split=pd.read_csv('City_split.csv')


### Sample data to be shared in automailer (Table 1)

In [65]:
SH

,SH,0 to 1,1 to 2,2 to 3,3 to 4,4 to 5,5 to 6,6 to 7,7 to 8,8 to 9,9 to 10,Days10+,Total
0,3PL_Forward,1035,603,463,323,233,118,27,7,4,5,27,2845
1,ML_Forward,399,83,44,26,25,14,11,7,4,4,31,648
2,ML_Transport,692,487,355,308,297,216,218,205,128,106,366,3378
3,MarketPlace_Not_Packed,16,4,0,0,0,0,0,0,0,0,1,21
4,Non-Returnable,3566,1862,807,541,398,287,214,154,135,100,402,8466
5,Others,944,1155,773,584,498,389,408,281,204,153,1314,6703
6,Reverse_3PL,1137,591,328,255,104,45,48,26,13,15,48,2610
7,Reverse_ML,365,108,19,30,25,11,16,17,5,2,45,643
8,Warehouse,24,20,15,15,35,21,32,16,14,22,132,346
Grand Total,3PL_ForwardML_ForwardML_TransportMarketPlace_N...,8178,4913,2804,2082,1615,1101,974,713,507,407,2366,25660


In [66]:
SH_1=SH.set_index('SH')
SH_1.rename(index={(SH_1.index.values[9]):'Grand Total'},inplace=True)
SH_1=SH_1.loc[['MarketPlace_Not_Packed', 'Warehouse','ML_Forward','ML_Transport','3PL_Forward','Reverse_ML','Reverse_3PL','Non-Returnable','Others', 'Grand Total']]
SH_1=SH_1.reset_index()

SH_Open_1=SH_Open.set_index('SH')
SH_Open_1.rename(index={(SH_Open_1.index.values[9]):'Grand Total'},inplace=True)
SH_Open_1=SH_Open_1.loc[['MarketPlace_Not_Packed', 'Warehouse','ML_Forward','ML_Transport','3PL_Forward','Reverse_ML','Reverse_3PL','Non-Returnable','Others', 'Grand Total']]
SH_Open_1=SH_Open_1.reset_index()

In [67]:
from datetime import date
today=date.today()
today=today.strftime("%d-%b-%Y")

### Write all the dataframes (i.e. teamwise reports) to Excel file (to be attached in automailer)

In [68]:
import pandas as pd
exclefilename="Task_Pendency_Report "+today+".xlsx"
writer = pd.ExcelWriter(exclefilename, engine='xlsxwriter')


# Write each dataframe to a different worksheet. 
SH_1.to_excel(writer, sheet_name='SH',index=False)
SH_Open_1.to_excel(writer, sheet_name='SH',index=False, startcol=0,startrow=13)
Incient_category.to_excel(writer, sheet_name='Incient_category',index=False)
Task_category.to_excel(writer, sheet_name='Task_category',index=False)
City_split.to_excel(writer, sheet_name='City_split',index=False)
Final_8.to_excel(writer, sheet_name='Raw Data',index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [70]:
    recipients= ['List of reciepients'
    ]

### Send automailer with summary and details excel report

In [71]:
    import mimetypes
    from email.mime.multipart import MIMEMultipart
    from email import encoders
    from email.message import Message
    from email.mime.audio import MIMEAudio
    from email.mime.base import MIMEBase
    from email.mime.image import MIMEImage
    from email.mime.text import MIMEText
    import datetime
    import sys
    import smtplib

    print ("Checking for Internet Connection\n")
    username = "username"
    password = "password"
    server = smtplib.SMTP("smtp.gmail.com:587")
    server.starttls()
    server.login(username,password)
    emailfrom = "email id"


    emailto=", ".join(recipients)

    msg = MIMEMultipart('alternative')


    msg["From"] = emailfrom
    msg["To"] = emailto
    msg["Subject"] = "CC Task Pendency Report- "+today


    part2 = MIMEBase('application', "octet-stream")

    part2.set_payload(open("Task_Pendency_Report "+today+".xlsx", "rb").read())
    encoders.encode_base64(part2)

    part2.add_header('Content-Disposition', "attachment", filename="Task_Pendency_Report "+today+".xlsx")
    msg.attach(part2)

    import io
    str_io = io.StringIO()
    SH_1.to_html(buf=str_io, classes='table table-striped', index = False)
    html_str1 = str_io.getvalue()

    str_io4 = io.StringIO()
    SH_Open_1.to_html(buf=str_io4, classes='table table-striped',index = False)
    html_str4 = str_io4.getvalue()

    str_io2 = io.StringIO()
    Incient_category_1.to_html(buf=str_io2, classes='table table-striped',index = False)
    html_str2 = str_io2.getvalue()

    str_io3 = io.StringIO()
    Task_category_1.to_html(buf=str_io3, classes='table table-striped',index = False)
    html_str3 = str_io3.getvalue()



    email_content = """
    <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
    <title>test</title>
    <style type="text/css" media="screen">
    table{
    background-color: #f7fdff;
    empty-cells:hide;
        border-collapse: collapse;
        border-spacing: 2px;  
    }
    th  {background-color:#586266; color:white;text-align: center;}
    td {
        text-align: center;

    }

    </style>
    </head>
    <body>

    <p>

    >>>>>>>>>>AUTOGENERATED MAIL<<<<<<<<<<>>>>>>>>>><<<<<<<<<<
    <br>
    <br>

    <strong> Hi All, 
    <br>

    Please find the updated Task Pendency report for 
            """+today+""".



    </p>

    <br>

    <b>1. Overall Pendency:</b>
    <br>


    <br>

    """+ html_str1 +  """

    <br>

    <b>1.1 Overall Pendency (Task Status - Open):</b>
    <br>
    <br>
    """+ html_str4 +  """

    <br>

    <b>2. Top 10 Customer's Issue:</b>
    <br>
    <br>
    """+ html_str2 +  """

    <br>

    <b>3. Top 10 Task Category:</b>
    <br>
    <br>
    """+ html_str3 +  """

    <br>
    <br>
    <br>
    <br>

    </body>
    """

    HTML_BODY = MIMEText(email_content.encode('utf-8'), 'html','utf-8')
    msg.attach(HTML_BODY)
    server = smtplib.SMTP("smtp.gmail.com:587")
    server.starttls()
    server.login(username,password)
    print("Sending...")
    server.sendmail(emailfrom, recipients, msg.as_string())

    server.quit()
    print("Sent")

Checking for Internet Connection

Sending...
Sent


### Send automailer to Escalations team sepaartely for high priority issues

In [72]:
Ed_1=Final_8[(~Final_8['escalated_to'].isnull())
           &((~Final_8['SH'].isin(['Others','Non-Returnable']))
               |(Final_8['Category Level 3'].isin(['Not Packed','Not Packed_WH'])))][['SH','Category Level 1','Category Level 2',
                       'Category Level 3','escalated_to','escalated_on','Order_ID','Task ID','Task Status','Tracking Number','Entity_Id','Task Created Date','Ageing_in_days',
                       'Ageing_bucket','zipcode','packet_return_id','courier_code','status_code','source_wh_id','city','tracking_number',
                       'DC','assignment_date','incident','queue_name','Intercation_count_call','inapp_interactions','task_queue',
                       'orderdetails_returnid','Total_interactions','address_type','shipment_status','dc_inscanned_date','PPMP/Vendor_Flex',
                       'Issue Type 1','Issue Type 2','Issue Type 3']]

In [73]:
t=Ed_1["escalated_on"].tolist()

from datetime import datetime
# In[20]:


#calculate ageing
i=0
length=len(Ed_1["escalated_on"])
ageing_in_hours=[]
c=datetime.now()
while i < length:
    j=(c-datetime.strptime(str(pd.to_datetime(t[i])),"%Y-%m-%d %H:%M:%S") ).total_seconds()/(86400)
    ageing_in_hours.append(round(j,2))
    i+=1
Ed_1["Ageing_from_escalation"]=ageing_in_hours


Ed_1.loc[(Ed_1['Ageing_from_escalation']>10),'ED_bucket'] = 'Days10+'
Ed_1.loc[(Ed_1['Ageing_from_escalation']<=10),'ED_bucket'] = '9 to 10'
Ed_1.loc[(Ed_1['Ageing_from_escalation']<=9),'ED_bucket'] = '8 to 9'
Ed_1.loc[(Ed_1['Ageing_from_escalation']<=8),'ED_bucket'] = '7 to 8'
Ed_1.loc[(Ed_1['Ageing_from_escalation']<=7),'ED_bucket'] = '6 to 7'
Ed_1.loc[(Ed_1['Ageing_from_escalation']<=6),'ED_bucket'] = '5 to 6'
Ed_1.loc[(Ed_1['Ageing_from_escalation']<=5),'ED_bucket'] = '4 to 5'
Ed_1.loc[(Ed_1['Ageing_from_escalation']<=4),'ED_bucket'] = '3 to 4'
Ed_1.loc[(Ed_1['Ageing_from_escalation']<=3),'ED_bucket'] = '2 to 3'
Ed_1.loc[(Ed_1['Ageing_from_escalation']<=2),'ED_bucket'] = '1 to 2'
Ed_1.loc[(Ed_1['Ageing_from_escalation']<=1),'ED_bucket'] = '0 to 1'


In [74]:
Task_category_ED=Ed_1.pivot_table(index=['Category Level 3'], values=['Task ID'], columns='Ageing_bucket', aggfunc=pd.Series.nunique).fillna(0).astype(np.int64)
Task_category_ED.columns=Task_category_ED.columns.get_level_values(1)
Task_category_ED=Task_category_ED.rename_axis(['Category Level 3']).reset_index(level=0)
Task_category_ED['Total']=Task_category_ED.sum(numeric_only=True, axis=1)
Task_category_ED.sort_values(by='Total', ascending=False,inplace=True)
Task_category_ED.to_csv('Task_category_ED.csv', index=False)
Task_category_ED=pd.read_csv('Task_category_ED.csv')
Task_category_ED_1=Task_category_ED[~Task_category_ED['Category Level 3'].isin(['Exchange Approvals', "Pick up Approvals'"])][:10]

In [75]:
SH_ED=Ed_1.pivot_table(index=['escalated_to'], values=['Task ID'], columns='Ageing_bucket', aggfunc=pd.Series.nunique).fillna(0).astype(np.int64)
SH_ED.columns=SH_ED.columns.get_level_values(1)
SH_ED=SH_ED.rename_axis(['Task_Queue']).reset_index(level=0)
SH_ED['Total']=SH_ED.sum(numeric_only=True, axis=1)
SH_ED.to_csv('SH_ED_ED.csv', index=False)
SH_ED=pd.read_csv('SH_ED_ED.csv')
SH_ED.loc['Grand Total']=SH_ED.sum( axis=0)
SH_ED=SH_ED.set_index('Task_Queue')
SH_ED.rename(index={(SH_ED.index.values[len(SH_ED)-1]):'Grand Total'},inplace=True)
SH_ED.reset_index(inplace=True)

In [76]:
SH_ED_2=Ed_1.pivot_table(index=['SH'], values=['Task ID'], columns='ED_bucket', aggfunc=pd.Series.nunique).fillna(0).astype(np.int64)
SH_ED_2.columns=SH_ED_2.columns.get_level_values(1)
SH_ED_2=SH_ED_2.rename_axis(['SH']).reset_index(level=0)
SH_ED_2['Total']=SH_ED_2.sum(numeric_only=True, axis=1)
SH_ED_2.to_csv('SH_ED_2.csv', index=False)
SH_ED_2=pd.read_csv('SH_ED_2.csv')
SH_ED_2.loc['Grand Total']=SH_ED_2.sum( axis=0)
SH_ED_2=SH_ED_2.set_index('SH')
SH_ED_2.rename(index={(SH_ED_2.index.values[len(SH_ED_2)-1]):'Grand Total'},inplace=True)
SH_ED_2.reset_index(inplace=True)

In [77]:
SH_ED_1=Ed_1.pivot_table(index=['escalated_to'], values=['Task ID'], columns='ED_bucket', aggfunc=pd.Series.nunique).fillna(0).astype(np.int64)
SH_ED_1.columns=SH_ED_1.columns.get_level_values(1)
SH_ED_1=SH_ED_1.rename_axis(['Task_Queue']).reset_index(level=0)
SH_ED_1['Total']=SH_ED_1.sum(numeric_only=True, axis=1)
SH_ED_1.to_csv('SH_ED_1.csv', index=False)
SH_ED_1=pd.read_csv('SH_ED_1.csv')
SH_ED_1.loc['Grand Total']=SH_ED_1.sum( axis=0)
SH_ED_1=SH_ED_1.set_index('Task_Queue')
SH_ED_1.rename(index={(SH_ED_1.index.values[len(SH_ED_1)-1]):'Grand Total'},inplace=True)
SH_ED_1.reset_index(inplace=True)

In [78]:
today_ED=str(date.today())

In [80]:
exclefilename="ED_Pendency "+today_ED+".xlsx"
writer = pd.ExcelWriter(exclefilename, engine='xlsxwriter')
SH_ED.to_excel(writer, sheet_name='Summary',index=False,startcol=0,startrow=2)
worksheet=writer.sheets['Summary']
worksheet.write('A1', 'Ageing_from_creation')
worksheet.write('A11', 'Ageing_from_escalation')

SH_ED_1.to_excel(writer, sheet_name='Summary',index=False,startcol=0,startrow=12)
SH_ED_2.to_excel(writer, sheet_name='Stakeholder_Cut',index=False)
Task_category_ED.to_excel(writer, sheet_name='Task_category_ED',index=False)
Ed_1.to_excel(writer, sheet_name='Raw Data',index=False)


# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [81]:
recipients_2=['List of recipients'']

In [83]:
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
import datetime
import sys
import smtplib

print ("Checking for Internet Connection\n")
username = "username"
password = "pswrd"
server = smtplib.SMTP("smtp.gmail.com:587")
server.starttls()
server.login(username,password)
emailfrom = "cemail id"

emailto=", ".join(recipients_2)
msg = MIMEMultipart('alternative')


msg["From"] = emailfrom
msg["To"] = emailto
msg["Subject"] = "SCM ED Task Pendency Report- "


part2 = MIMEBase('application', "octet-stream")

part2.set_payload(open("ED_Pendency "+today_ED+".xlsx", "rb").read())
encoders.encode_base64(part2)

part2.add_header('Content-Disposition', "attachment", filename="ED_Pendency "+today_ED+".xlsx")
msg.attach(part2)


import io
str_io = io.StringIO()
SH_ED_2.to_html(buf=str_io, classes='table table-striped', index = False)
html_str3 = str_io.getvalue()

str_io = io.StringIO()
SH_ED.to_html(buf=str_io, classes='table table-striped', index = False)
html_str1 = str_io.getvalue()

str_io = io.StringIO()
SH_ED_1.to_html(buf=str_io, classes='table table-striped', index = False)
html_str2 = str_io.getvalue()

str_io4 = io.StringIO()
Task_category_ED_1.to_html(buf=str_io4, classes='table table-striped',index = False)
html_str4 = str_io4.getvalue()


email_content = """
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<title>test</title>
<style type="text/css" media="screen">
table{
background-color: #f7fdff;
empty-cells:hide;
   border-collapse: collapse;
   border-spacing: 2px;
   padding: 15px;
}
th  {background-color:#586266; color:white;text-align: center;}
td {
   text-align: center;
   padding: 7px;
   
 
}

</style>
</head>
<body>

<p>

>>>>>>>>>>AUTOGENERATED MAIL<<<<<<<<<<>>>>>>>>>><<<<<<<<<<
<br>
<br>

<strong> Hi All,
<br>

Please find the Updated SCM ED Task Pendency for
       """+today+""".
       
</p>

<br>

<b>1. Overall SH ED Pendency (from escalation):</b>
<br>


<br>

"""+ html_str3 +  """

<br>
<b>2. Overall ED Pendency (from task creation):</b>
<br>


<br>

"""+ html_str1 +  """

<br>
<b>3. ED Pendency (from escalation):</b>
<br>


<br>

"""+ html_str2 +  """

<br>

<b>4. Top 10 Task Categories:</b>
<br>
<br>
"""+ html_str4 +  """

<br>

<br>


<br>
<br>
<br>
<br>

</body>
"""

HTML_BODY = MIMEText(email_content.encode('utf-8'), 'html','utf-8')

msg.attach(HTML_BODY)


server = smtplib.SMTP("smtp.gmail.com:587")

server.starttls()

server.login(username,password)
print("Sending...")
server.sendmail(emailfrom, recipients_2, msg.as_string())


server.quit()
print("Sent")

Checking for Internet Connection

Sending...
Sent
